In [1]:
from utils import roar_code_path as project_code_path

In [2]:
import os
import numpy as np
import xarray as xr
from glob import glob
from utils import roar_code_path as project_code_path
from utils import roar_data_path as project_data_path
from utils import map_store_names
from utils import get_unique_loca_metrics
import dask

In [3]:
from utils import gev_metric_ids

In [4]:
import pandas as pd
import xarray as xr

In [5]:
from dask.distributed import LocalCluster
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 1
Total threads: 1,Total memory: 30.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35287,Workers: 1
Dashboard: /proxy/8787/status,Total threads: 1
Started: Just now,Total memory: 30.00 GiB
Comm: tcp://127.0.0.1:42765,Total threads: 1
Dashboard: /proxy/37645/status,Memory: 30.00 GiB
Nanny: tcp://127.0.0.1:41229,


In [7]:
params_in = xr.open_dataset('/storage/group/pches/default/users/dcl5300/conus_comparison_lafferty-etal-2024/extreme_value/original_grid/max_pr/GARD-LENS_canesm5_r1i1p1f1_ssp370_1950-2100')

In [13]:
import gev_nonstat_utils as gevns
import importlib
importlib.reload(gevns)

<module 'gev_nonstat_utils' from '/storage/work/dcl5300/current_projects/conus_comparison_lafferty-etal-2024/src/gev_nonstat_utils.py'>

In [16]:
gevns.fit_ns_gev_single(
    ensemble='GARD-LENS',
    gcm='canesm5',
    member='r1i1p1f1',
    ssp='ssp370',
    metric_id='max_pr',
    bootstrap=True,
    n_boot=100,
)

In [17]:
xr.open_dataset('/storage/group/pches/default/users/dcl5300/conus_comparison_lafferty-etal-2024/extreme_value/original_grid/max_pr/GARD-LENS_canesm5_r1i1p1f1_ssp370_1950-2100_nonstat_mle_100boot.nc')

<xarray.Dataset> Size: 3kB
Dimensions:                  (gcm: 1, member: 1, ssp: 1, ensemble: 1,
                              quantile: 2, lat: 2, lon: 2, time: 7, time_diff: 1)
Coordinates:
  * gcm                      (gcm) <U7 28B 'CanESM5'
  * member                   (member) <U8 32B 'r1i1p1f1'
  * ssp                      (ssp) <U6 24B 'ssp370'
  * ensemble                 (ensemble) <U9 36B 'GARD-LENS'
  * lat                      (lat) float32 8B 37.62 37.75
  * lon                      (lon) float32 8B -112.4 -112.2
  * time                     (time) int64 56B 1950 1975 2000 2025 2050 2075 2100
  * time_diff                (time_diff) <U9 36B '1975-2075'
  * quantile                 (quantile) <U4 32B 'q025' 'q975'
Data variables: (12/16)
    loc_intcp                (gcm, member, ssp, ensemble, quantile, lat, lon) float64 64B ...
    loc_trend                (gcm, member, ssp, ensemble, quantile, lat, lon) float64 64B ...
    scale                    (gcm, member, ssp, ensemble, quantile, lat, lon) float64 64B ...
    shape                    (gcm, member, ssp, ensemble, quantile, lat, lon) float64 64B ...
    10yr_return_level        (gcm, member, ssp, ensemble, quantile, time, lat, lon) float64 448B ...
    25yr_return_level        (gcm, member, ssp, ensemble, quantile, time, lat, lon) float64 448B ...
    ...                       ...
    50yr_return_level_diff   (gcm, member, ssp, ensemble, quantile, time_diff, lat, lon) float64 64B ...
    100yr_return_level_diff  (gcm, member, ssp, ensemble, quantile, time_diff, lat, lon) float64 64B ...
    10yr_return_level_chfc   (gcm, member, ssp, ensemble, quantile, time_diff, lat, lon) float64 64B ...
    25yr_return_level_chfc   (gcm, member, ssp, ensemble, quantile, time_diff, lat, lon) float64 64B ...
    50yr_return_level_chfc   (gcm, member, ssp, ensemble, quantile, time_diff, lat, lon) float64 64B ...
    100yr_return_level_chfc  (gcm, member, ssp, ensemble, quantile, time_diff, lat, lon) float64 64B ...

In [34]:
out = gevns.fit_ns_gev_xr_bootstrap(
    params_in = params_in.isel(lat=slice(100,102), lon=slice(100,102)),
    metric_id='max_pr',
    expected_length=151,
    starting_year=1950,
    fit_method="mle",
    periods_for_level=[10, 25, 50, 100],
    return_period_years=[1950, 1975, 2000, 2025, 2050, 2075, 2100],
    return_period_diffs=[(2075, 1975)],
    n_boot=2,
    return_samples=False)

In [35]:
out

<xarray.Dataset> Size: 3kB
Dimensions:                  (quantile: 2, lat: 2, lon: 2, time: 7, time_diff: 1)
Coordinates:
  * lat                      (lat) float32 8B 37.62 37.75
  * lon                      (lon) float32 8B -112.4 -112.2
  * time                     (time) int64 56B 1950 1975 2000 2025 2050 2075 2100
  * time_diff                (time_diff) object 8B '1975-2075'
  * quantile                 (quantile) <U4 32B 'q025' 'q975'
Data variables: (12/16)
    loc_intcp                (quantile, lat, lon) float64 64B 26.2 ... 26.72
    loc_trend                (quantile, lat, lon) float64 64B 0.005675 ... 0....
    scale                    (quantile, lat, lon) float64 64B 9.134 ... 10.32
    shape                    (quantile, lat, lon) float64 64B -0.142 ... 0.03281
    10yr_return_level        (quantile, time, lat, lon) float64 448B 53.14 .....
    25yr_return_level        (quantile, time, lat, lon) float64 448B 65.48 .....
    ...                       ...
    50yr_return_level_diff   (quantile, time_diff, lat, lon) float64 64B -7.6...
    100yr_return_level_diff  (quantile, time_diff, lat, lon) float64 64B -7.6...
    10yr_return_level_chfc   (quantile, time_diff, lat, lon) float64 64B 0.87...
    25yr_return_level_chfc   (quantile, time_diff, lat, lon) float64 64B 0.89...
    50yr_return_level_chfc   (quantile, time_diff, lat, lon) float64 64B 0.90...
    100yr_return_level_chfc  (quantile, time_diff, lat, lon) float64 64B 0.91...

In [17]:
params_in['quantile'] = ['1']

In [5]:
import gev_nonstat_utils as gevns
import importlib
importlib.reload(gevns)

<module 'gev_nonstat_utils' from '/storage/work/dcl5300/current_projects/conus_comparison_lafferty-etal-2024/src/gev_nonstat_utils.py'>

In [6]:
params_in = np.array([10, 0.5, 10, -0.2])
expected_length = 151
n_boot = 100
fit_method = 'mle'

In [8]:
%%timeit
gevns._gev_fit_parametric_bootstrap_1d_nonstationary(
    params_in,
    expected_length,
    n_boot,
    fit_method
)

1.01 s ± 17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
gevns._gev_fit_parametric_bootstrap_1d_nonstationary(
    params_in,
    expected_length,
    n_boot,
    fit_method
)

974 ms ± 13.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
974e-3*224*464/60/60/20

1.406023111111111

In [9]:
474*949/60/60/20

6.247583333333333

In [10]:
596*1385/60/60/20

11.464722222222221

In [ ]:
%%time
out = gevns.fit_ns_gev_xr_bootstrap(params_in = params_in.isel(lat=slice(100,120), lon=slice(200,220)),
                                  metric_id='max_pr',
                                  starting_year = 1950,
                                  expected_length = 151,
                                    n_boot = 100,
                                  periods_for_level=[10, 25, 50, 100],
                                  return_period_years=[1950, 1975, 2000, 2025, 2050, 2075, 2100],
                                  return_period_diffs=[(2075, 1975)])

In [43]:
%%time
out = gevns.fit_ns_gev_xr_bootstrap(params_in = params_in.isel(lat=slice(100,120), lon=slice(200,220)),
                                  metric_id='max_pr',
                                  starting_year = 1950,
                                  expected_length = 151,
                                    n_boot = 100,
                                  periods_for_level=[10, 25, 50, 100],
                                  return_period_years=[1975, 2000, 2025, 2050, 2075, 2100],
                                  return_period_diffs=[(2075, 1975)])

CPU times: user 9.63 s, sys: 622 ms, total: 10.3 s
Wall time: 2min 12s


In [44]:
2/(20*20)*(224*464)/60

8.661333333333335